In [2]:
import pandas as pd
from rich import print as rprint
url = 'https://github.com/mattharrison/datasets/raw/master/data/vehicles.csv.zip'
df = pd.read_csv(url, dtype_backend='pyarrow', engine='pyarrow')
city_mpg = df.city08
highway_mpg = df.highway08
len(dir(city_mpg))

420

`.apply` applies function to an entire series or element-wise to every value. If it does the latter, you lose the optimized and fast storage of `numpy` because you pull data into Python. It breaks out of the fast vectorized code paths we can leverage in pandas

In [3]:
%%timeit
def gt20(val):
    return val > 20

city_mpg.apply(gt20)


3 ms ± 16.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Use the broadcasted `.gt` method, and it runs almost 50 times faster

In [4]:
%timeit city_mpg.gt(20)

28.8 μs ± 1.3 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Show the top five makes and label everything else *other*

In [5]:
make = df.make
make

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [6]:
make.value_counts()

make
Chevrolet                           4003
Ford                                3371
Dodge                               2583
GMC                                 2494
Toyota                              2071
                                    ... 
Grumman Allied Industries              1
Environmental Rsch and Devp Corp       1
General Motors                         1
Goldacre                               1
Isis Imports Ltd                       1
Name: count, Length: 136, dtype: int64[pyarrow]

In [7]:
top5 = make.value_counts().index[:5]
rprint(top5)


Index(['Chevrolet', 'Ford', 'Dodge', 'GMC', 'Toyota'], dtype='string[pyarrow]', name='make')

In [8]:
%%timeit
def generalize_top5(val):
    if val in top5:
        return val
    return 'Other'
make.apply(generalize_top5)

19.4 ms ± 341 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Using the .where method
This method takes a boolean array to mark where the condition is `True`. Keeps values from the series it is called on where boolean array is `True`, and if the boolean array is `False`, it uses the value of the second parameter, `Other`

In [9]:
rprint(make.where(make.isin(top5), other='Other'))

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: string[pyarrow]

In this case the `.where` method is about fifteen times faster, for this data, than using `.apply`

In [10]:
%timeit make.where(make.isin(top5),other='Other')

727 μs ± 2.97 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The complement of the `where` method is the `mask` method. Wherever the condition is `False` it keeps the original values; if it is `True` it replaces the value with the `other` parameter. The tilde, `~`, performs an inversion of the boolean array, switching all `True` values to `False` and vice versa

In [11]:
make.mask(~make.isin(top5),other='Other')

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: string[pyarrow]

Broadcast `numpy` function to entire series. `np.log` is a universal function ( *ufunc* ) and works on arrays.

In [12]:
import numpy as np

np.__version__

%timeit city_mpg.apply(np.log)

125 μs ± 66.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Compare that with Python's math module:

In [13]:
import math
%timeit city_mpg.apply(math.log)

2.72 ms ± 2.88 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


keep the top five makes and get the top 10 for the remainder of the top ten, other for the rest

In [14]:
vc = make.value_counts()
top5 = vc.index[:5] # start at 0 stop at 5
top10 = vc.index[:10] # start at 0 stop at 10

def generalize(val):
    if val in top5:
        return val
    elif val in top10:
        return 'Top10'
    else:
        return 'Other'
    
make.apply(generalize)

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

### `.case_when` method
replace values where conditions are `True`
provide list of tuples for `caselist` parameter:
1. boolean array or function taking series and returning boolean array
2. values when the boolean array is `True`
3. series of boolean `True` values with `Other` as replacement value


In [15]:
(make
 .case_when(caselist=[(make.isin(top5),make),
                      (make.isin(top10),'Top10'),
                      (pd.Series(True,index=make.index), 'Other')]))

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

### Alternatively, with `.where`
the same thing can be achieved....

In [16]:
(make
 .where(make.isin(top5), 'Top10')
 .where(make.isin(top10), 'Other'))

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: string[pyarrow]

### The *cylinders* column has missing values. 


In [17]:
cyl = df.cylinders
(cyl
 .isna()
 .sum())


np.int64(206)

If we index `.loc` with a boolean array, it returns the rows where the boolean array is `True`

In [18]:
missing = cyl.isna()
make.loc[missing]

7138     Nissan
7139     Toyota
8143     Toyota
8144       Ford
8146       Ford
          ...  
34563     Tesla
34564     Tesla
34565     Tesla
34566     Tesla
34567     Tesla
Name: make, Length: 206, dtype: string[pyarrow]

*index operation* is performed with square brackets `[ ]`
If we index `.loc` with a boolean array, it returns rows where the boolean array is `True`
The `.fillna` method allows user to specify replacement value for any missing data
Operations return a new Series

In [19]:
cyl[cyl.isna()]

7138     <NA>
7139     <NA>
8143     <NA>
8144     <NA>
8146     <NA>
         ... 
34563    <NA>
34564    <NA>
34565    <NA>
34566    <NA>
34567    <NA>
Name: cylinders, Length: 206, dtype: int64[pyarrow]

In [20]:
cyl.fillna(0).loc[7136:7141]

7136    6
7137    6
7138    0
7139    0
7140    6
7141    6
Name: cylinders, dtype: int64[pyarrow]

In [21]:
temp = pd.Series([32,40,None,42,39,32], dtype='float[pyarrow]')
temp

0    32.0
1    40.0
2    <NA>
3    42.0
4    39.0
5    32.0
dtype: float[pyarrow]

### `.interpolate()` ordered data, like temperatures, to replace missing values

In [22]:
temp.interpolate()

0    32.0
1    40.0
2    41.0
3    42.0
4    39.0
5    32.0
dtype: float[pyarrow]

### `.clip()` outliers in data
The first 447 entries in `city_mpg` range from 9 - 31. Trim these values to be between 5th (11.0) and 95th (27.0) quantile

In [23]:
(city_mpg
 .loc[:446]
 .clip(lower=city_mpg.quantile(.05),
       upper=city_mpg.quantile(.95)))

0      19
1      11
2      23
3      11
4      17
       ..
442    15
443    15
444    15
445    15
446    27
Name: city08, Length: 447, dtype: int64[pyarrow]

### Sorting values vs Sorting the index
the `.sort_values()` method will sort values in ascending order and also rearrange the index accordingly:

In [24]:
city_mpg.sort_values()

7901       6
21060      6
34557      6
35887      6
37161      6
        ... 
34563    138
34564    140
31256    150
32599    150
33423    150
Name: city08, Length: 41144, dtype: int64[pyarrow]

Because of index alignment, you can still do math operations on a sorted series

In [25]:
(city_mpg.sort_values() + highway_mpg) / 2

0        22.0
1        11.5
2        28.0
3        11.0
4        20.0
         ... 
41139    22.5
41140    24.0
41141    21.0
41142    21.0
41143    18.5
Length: 41144, dtype: double[pyarrow]

In [26]:
rprint(city_mpg)
rprint(city_mpg.sort_values())
rprint(city_mpg.sort_values().sort_index())

0        19
1         9
2        23
3        10
4        17
         ..
41139    19
41140    20
41141    18
41142    18
41143    16
Name: city08, Length: 41144, dtype: int64[pyarrow]

7901       6
21060      6
34557      6
35887      6
37161      6
        ... 
34563    138
34564    140
31256    150
32599    150
33423    150
Name: city08, Length: 41144, dtype: int64[pyarrow]

0        19
1         9
2        23
3        10
4        17
         ..
41139    19
41140    20
41141    18
41142    18
41143    16
Name: city08, Length: 41144, dtype: int64[pyarrow]

The `.drop_duplicates()` method will remove values appearing more than once...you can keep the first, or last, or `False` to remove all duplicates, including the initial value

In [27]:
city_mpg.drop_duplicates()

0         19
1          9
2         23
3         10
4         17
        ... 
34364    127
34409    114
34564    140
34565    115
34566    104
Name: city08, Length: 105, dtype: int64[pyarrow]

### `.rank()`
Returns a Series that keeps the original index but uses the ranks of values from the original Series. customize ranking with the `method` parameter. By default, if two values are the same, their rank will be the average of the positions they take

specify `min` to put equal values in the same rank, but the next rank will be the number of values that were equal

In [28]:
city_mpg.rank()

0        27060.5
1          235.5
2        35830.0
3          607.5
4        19484.0
          ...   
41139    27060.5
41140    29719.5
41141    23528.0
41142    23528.0
41143    15479.0
Name: city08, Length: 41144, dtype: double[pyarrow]

In [29]:
city_mpg.rank(method='min')

0        25555
1          136
2        35119
3          336
4        17467
         ...  
41139    25555
41140    28567
41141    21502
41142    21502
41143    13492
Name: city08, Length: 41144, dtype: uint64[pyarrow]

Or use the `dense` option to list ranks consecutively

In [30]:
city_mpg.rank(method='dense')

0        14
1         4
2        18
3         5
4        12
         ..
41139    14
41140    15
41141    13
41142    13
41143    11
Name: city08, Length: 41144, dtype: uint64[pyarrow]

### Using the `cut` function to create bins of equal width

In [31]:
pd.cut(city_mpg,10)

0        (5.856, 20.4]
1        (5.856, 20.4]
2         (20.4, 34.8]
3        (5.856, 20.4]
4        (5.856, 20.4]
             ...      
41139    (5.856, 20.4]
41140    (5.856, 20.4]
41141    (5.856, 20.4]
41142    (5.856, 20.4]
41143    (5.856, 20.4]
Name: city08, Length: 41144, dtype: category
Categories (10, interval[float64, right]): [(5.856, 20.4] < (20.4, 34.8] < (34.8, 49.2] < (49.2, 63.6] ... (92.4, 106.8] < (106.8, 121.2] < (121.2, 135.6] < (135.6, 150.0]]

use `cut` to create specific size bins. Note the bins have a half-open interval. They do not have the start value but do include the end value. 
If `city_mpg` had values with 0 or above 150, they would be missing after binning the series

In [32]:
pd.cut(city_mpg, [0,10,20,40,70,150])

0        (10, 20]
1         (0, 10]
2        (20, 40]
3         (0, 10]
4        (10, 20]
           ...   
41139    (10, 20]
41140    (10, 20]
41141    (10, 20]
41142    (10, 20]
41143    (10, 20]
Name: city08, Length: 41144, dtype: category
Categories (5, interval[int64, right]): [(0, 10] < (10, 20] < (20, 40] < (40, 70] < (70, 150]]

### Bin data with quantiles using `qcut`
Ten bins with approximately equal numbers of entries in each bin (rather than each bin width being the same)

In [33]:
pd.qcut(city_mpg,10)

0         (18.0, 20.0]
1        (5.999, 13.0]
2         (21.0, 24.0]
3        (5.999, 13.0]
4         (16.0, 17.0]
             ...      
41139     (18.0, 20.0]
41140     (18.0, 20.0]
41141     (17.0, 18.0]
41142     (17.0, 18.0]
41143     (15.0, 16.0]
Name: city08, Length: 41144, dtype: category
Categories (10, interval[float64, right]): [(5.999, 13.0] < (13.0, 14.0] < (14.0, 15.0] < (15.0, 16.0] ... (18.0, 20.0] < (20.0, 21.0] < (21.0, 24.0] < (24.0, 150.0]]

In [34]:
pd.qcut(city_mpg,10,labels=list(range(1,11)))

0        7
1        1
2        9
3        1
4        5
        ..
41139    7
41140    7
41141    6
41142    6
41143    4
Name: city08, Length: 41144, dtype: category
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]

### Indexing Operations

In [36]:
city2 = city_mpg.rename(make.to_dict())
city2

Alfa Romeo    19
Ferrari        9
Dodge         23
Dodge         10
Subaru        17
              ..
Subaru        19
Subaru        20
Subaru        18
Subaru        18
Subaru        16
Name: city08, Length: 41144, dtype: int64[pyarrow]

The `.rename` method accepts a series, a scalar, or a function that takes an old label and returns a new label or sequence. When we pass in a series, and the index values are the same, the values from the series that we passed in are used as the index. Careful though! if you pass a scalar value (a single string) innto `.rename`, the index will stay the same, but the `.name` attribute of the series will update

In [39]:
city2.index
city2.rename(make)

Alfa Romeo    19
Ferrari        9
Dodge         23
Dodge         10
Subaru        17
              ..
Subaru        19
Subaru        20
Subaru        18
Subaru        18
Subaru        16
Name: city08, Length: 41144, dtype: int64[pyarrow]

### Resetting the Index
set the index to monotonic increasing, that is, unique integers starting at zero. Returns a dataframe

In [52]:
print(city2.reset_index())

            index  city08
0      Alfa Romeo      19
1         Ferrari       9
2           Dodge      23
3           Dodge      10
4          Subaru      17
...           ...     ...
41139      Subaru      19
41140      Subaru      20
41141      Subaru      18
41142      Subaru      18
41143      Subaru      16

[41144 rows x 2 columns]


Drop the current index and return a Series

In [53]:
city2.reset_index(drop=True)

0        19
1         9
2        23
3        10
4        17
         ..
41139    19
41140    20
41141    18
41142    18
41143    16
Name: city08, Length: 41144, dtype: int64[pyarrow]

`.loc` attribute deals with index `labels`. It allows you to pull out pieces of the series. You can pass:
1. A scalar value of one of the index labels
2. A list of index labels
3. A slice of labels (closed interval so it includes the stop value)
4. An index
5. A boolean array (same index labels as the series, but with True or False values)
6. A function that accepts a series and returns one of the above

If you pass in a scalar with the label of an index, and there are duplicate labels, it will return a series, but if there is only one value for that label, it will return a scalar.

In [54]:
city2.loc['Subaru']

Subaru    17
Subaru    21
Subaru    22
Subaru    19
Subaru    20
          ..
Subaru    19
Subaru    20
Subaru    18
Subaru    18
Subaru    16
Name: city08, Length: 885, dtype: int64[pyarrow]

In [55]:
city2.loc[['Fisker']]

Fisker    20
Name: city08, dtype: int64[pyarrow]

In [56]:
city2.loc[['Ferrari', 'Lamborghini']]

Ferrari         9
Ferrari        12
Ferrari        11
Ferrari        10
Ferrari        11
               ..
Lamborghini     6
Lamborghini     8
Lamborghini     8
Lamborghini     8
Lamborghini     8
Name: city08, Length: 357, dtype: int64[pyarrow]

`.loc` we can use a slice with string values. First, sort the index if slicing with duplicate index labels. When slicing with `.loc`, it follows the `closed interval`. The closed interval includes both the start index and the final index. This is different than Python's list slicing behavior which includes the starting index going up but excluding the final index. This is called, `half-open interval`

In [57]:
city2.loc["Ferrari":"Lamborghini"]

KeyError: "Cannot get left slice bound for non-unique label: 'Ferrari'"

In [ ]:
city2.sort_index().loc["Ferrari":"Lamborghini"]

Ferrari        10
Ferrari        13
Ferrari        13
Ferrari         9
Ferrari        10
               ..
Lamborghini    12
Lamborghini     9
Lamborghini     8
Lamborghini    13
Lamborghini     8
Name: city08, Length: 11210, dtype: int64[pyarrow]

All the labels in `city2` that start with `F` "going up" to `J`. 

In [ ]:
city2.sort_index().loc["F":"J"]

Federal Coach    15
Federal Coach    13
Federal Coach    13
Federal Coach    14
Federal Coach    13
                 ..
Isuzu            15
Isuzu            15
Isuzu            15
Isuzu            27
Isuzu            18
Name: city08, Length: 9040, dtype: int64[pyarrow]

If we index into `.loc` with a simple Index with only `Dodge` in it, we get back every value for that label. Using an index is useful if we want to align a series to a new index:

In [ ]:
idx = pd.Index(['Dodge'])
city2.loc[idx]

Dodge    23
Dodge    10
Dodge    12
Dodge    11
Dodge    11
         ..
Dodge    18
Dodge    17
Dodge    14
Dodge    14
Dodge    11
Name: city08, Length: 2583, dtype: int64[pyarrow]

Pass in a boolean array to `.loc`. The boolean array is a series with the same index labels as the series that you are manipulating. If you do an indexing operation off of `.loc` with a boolean array, it will return only the values where the boolean array was true.

In [58]:
mask = city2 > 50
mask

Alfa Romeo    False
Ferrari       False
Dodge         False
Dodge         False
Subaru        False
              ...  
Subaru        False
Subaru        False
Subaru        False
Subaru        False
Subaru        False
Name: city08, Length: 41144, dtype: bool[pyarrow]

In [59]:
city2.loc[mask]

Nissan     81
Toyota     81
Toyota     81
Ford       74
Nissan     84
         ... 
Tesla     140
Tesla     115
Tesla     104
Tesla      98
Toyota     55
Name: city08, Length: 236, dtype: int64[pyarrow]

### Use Function with .loc
Considering a 10% increase in cost over the last year...find all the new prices that are above $3 after inflation

In [61]:
cost = pd.Series([1.00, 2.25, 3.99, .99, 2.79], index=['Gum', 'Cookie', 'Melon', 'Roll', 'Carrots'])
inflation = 1.10
(cost.mul(inflation).loc[lambda s_:s_ > 3])

Melon      4.389
Carrots    3.069
dtype: float64

If I calculate the boolearn array before taking into account the inflation, in other words, using the old series instead of the chained intermediate values, the answer is wrong

In [62]:
cost = pd.Series([1.00, 2.25, 3.99, .99, 2.79], index=['Gum', 'Cookie', 'Melon', 'Roll', 'Carrots'])
inflation = 1.10
mask = cost > 3
(cost.mul(inflation).loc[mask])

Melon    4.389
dtype: float64

### .iloc Attribute
When slicing off of this attribute, we pull out items by index position. The `.iloc` attribute supports indexing with the following:
1. scalar index position (integer)
2. list of index positions
3. slice of positions (half-open interval, exclusive of stop index/value)
4. NumPy array (or Python list) of boolean values
5. Function that accepts a series and returns one of the above

In [63]:
city2.iloc[0]

19

Use negative indexing to pull out the last value; same as Python list

In [64]:
city2.iloc[-1]

16